### ```spark streaming```

### - 스트리밍 처리 but micro-batch 

### - micro-batch : 작은 시간 간격으로 일정한 batch size 로 묶어 거의 실시간으로 처리

### - 시간대 별로 data aggregate and analyze 가능.

### - checkpoint 를 사용하여 결함이 발생한 시점으로 돌아가 작업 가능.

### - kafka, aws kinesis, hdfs 확장가능

### - workflow

`input data -> spark streaming -> batch input data (d-streaming) -> spark engine -> processed data`

spark 의 데이터는 분산저장, 같은 연산도 여러번, 불변의 속성을 가지고 있다고 여러번 말했었다.

streaming 처리 시에도 일관된 spark 의 특징을 알 수 있다.

batch input data (d-streaming) 단계에서 데이터는 batch 단위로 분산 저장된다.

이 때 분산 시킨 batch 단위의 데이터들은 독립적인 RDD 로 처리된다.

그리고 위에서 설명했던 것처럼 시간의 구간을 나눠 데이터를 batch processing 하기 때문에

결함이 발생했다면, 다시 그 시점으로 돌아가 해당 부분부터의 처리를 할 수 있게 되는 것이다.

`Dstream -> RDD TIME 1 (0 ~ 1 sec data) -> spark engine -> RDD TIME 2 (1 ~ 2 sec data) -> spark engine -> .....`

위와 같은 방식으로 streaming 처리를 한다.

batch size 에 대해 주의해야하는 점은

-batch size 가 너무 작을 경우 overhead 를 증가시켜 클러스터의 resource 를 소모시켜 성능저하가 일어난다.

-batch size 가 너무 클 경우 latency 를 발생시켜 실시간의 의미를 퇴색시키고 메모리가 부족해진다.

적절한 batch size 를 설정하는 것도 중요하다.

그리고 present data 를 처리하기위해 previous data 의 정보가 필요할 때에는 window-operator 로 데이터를 묶어서 처리한다.

가령 RDD TIME 5 을 처리하기 위해 RDD TIME 3 이 필요하다면,

`(RDD TIME 1, RDD TIME 2, (RDD TIME 3), RDD TIME 4, RDD TIME 5)` 이런 식으로 window-operator 를 사용하여 처리할 수 있다. 

`() 는 윈도우를 표현한 것이고 위의 설명은 sliding window.`

일정 시간 간격으로 윈도우를 움직이면서 또는 일정 간격으로 batch size 를 생성해서 해당 기간의 data 를 처리할 수 있다. 

`(RDD TIME 1, RDD TIME 2, RDD TIME 3) -> result1, (RDD TIME 4, RDD TIME 5, RDD TIME 6) -> result2`

`() 는 윈도우를 표현한 것이고 위의 설명은 tumbling window. 각각의 RDD 들은 tumbling window 처리 후 새로운 RDD 로 합쳐짐.`

그리고 spark streaming 은 kafka 와 같이 사용이 가능한데,

spark.readStream.format('kafka')\
                .option('kafka.bootstrap_servers = ~')\
                .option('subscribe', 'topicname').load()

와 같이 kafka 의 producer 에 의해 저장된 topic 안의 message 를 spark 의 dataframe 으로 가져올 수 있다. 

kafka 역시 streaming processing 에 장점이 있는 툴이기 때문아닐까 한다.

그리고 spark 의 dataframe 으로 가져왔기 때문에 transform 을 사용할 수 있고,

그렇게 transform 된 데이터를 어떤 확장자로 저장할지, micro-batch 실행 간격을 어떻게 할지 등을 정할 수 있다.

그렇게 설정이 끝나면 .start() method 로 spark 에 streaming processing 을 시작할 수 있다.

-------------------------------------------------------------------------------------

개인적인 생각.

streaming 처리를 할 때 신경써야 할 부분들이 더 있는 것 같다.

위에서 말한 것처럼, micro-batch 를 이용한 batch 단위의 데이터들은 독립적인 RDD 로 처리되기 때문에 기본적으로 파티셔닝은 중요하지만

batch size, 스트리밍 처리 시간 조절 역시 신경을 써야하는게 맞는 것 같다.

보통 spark 에서 다루는 데이터의 볼륨은 매우 클 가능성이 높고, 그 큰 데이터들의 batch size 와 스트리밍 처리시간을 조절하지 않으면

파티셔닝은 적절했어도 매우 많은 양의 RDD 가 생성될 것이고, 그렇게 많아진 RDD 들을 처리하는 것은 매우 힘든 일이 될 것이다.

그 많아진 RDD 를 다시 하나의 RDD 로 묶는다는 생각은 RDD 의 태생을 부정한다.

그리고 그렇게 합친다 하더라도 shuffling 은 반드시 발생한다고 생각하고 어마어마한 resource 가 필요할 것이다.

오히려 RDD 장점이 단점으로 바뀌게 되는 일이 발생하지 않도록 신경써야 할 것 같다.